In [2]:
!pip -q install xgboost shap

import os, re, json, warnings, math, random
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict, Any

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    accuracy_score, precision_recall_fscore_support
)
from sklearn.model_selection import TimeSeriesSplit, KFold, RandomizedSearchCV, learning_curve

from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBRegressor
import shap
import joblib

# CONFIG
RANDOM_STATE = 42
random.seed(RANDOM_STATE)

TARGET = "SPI_smoothed"
DEFAULT_CSV_PATH = "/content/final_dataset.csv"

OUT_DIR = "/content/outputs"
MODEL_DIR = "/content/models"
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

DO_TUNE = True
N_ITER_TUNE = 20
CV_SPLITS = 5
SHAP_SAMPLE = 2000
HASH_DIM = 512
TOP_FEATS = 30

# Strategy to convert continuous SPI -> HighRisk=True
CLASS_THRESHOLD_STRATEGY = "median"   # median
CLASS_THRESHOLD_Q = 0.5

# per-vehicle thresholds
CLASS_THRESHOLD_PER_VEHICLE = True
MIN_TRAIN_RECORDS_PER_VEHICLE = 8     # if fewer than this in training, fall back to GLOBAL

warnings.filterwarnings(
    "ignore",
    message="This Pipeline instance is not fitted yet",
    category=FutureWarning
)
warnings.filterwarnings(
    "ignore",
    message="The NumPy global RNG was seeded by calling `np.random.seed`",
    category=FutureWarning
)

# UTILS
def build_timestamp(df: pd.DataFrame) -> pd.Series:
    ts = None
    if 'ts' in df.columns:
        t = pd.to_datetime(df['ts'], errors="coerce")
        if t.notna().any(): ts = t
    if ts is None and 'Datetime' in df.columns:
        t = pd.to_datetime(df['Datetime'], errors="coerce")
        if t.notna().any(): ts = t
    if ts is None and ('Date' in df.columns or 'Time' in df.columns):
        if 'Date' in df.columns and 'Time' in df.columns:
            t = pd.to_datetime(df['Date'].astype(str) + " " + df['Time'].astype(str), errors="coerce")
        else:
            t = pd.to_datetime(df.get('Date', pd.NaT), errors="coerce")
        ts = t
    if ts is None:
        ts = pd.Series([pd.NaT]*len(df), index=df.index)
    return (ts.astype('int64') // 10**9).astype('float')

def drop_slope_elevation(df: pd.DataFrame) -> pd.DataFrame:
    to_drop = [c for c in df.columns if ('slope' in c.lower()) or ('elev' in c.lower())]
    if to_drop:
        print(f"[Info] Dropping slope/elevation columns: {to_drop}")
    return df.drop(columns=to_drop, errors="ignore")

class DenseHashingVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=HASH_DIM, input_col='text'):
        self.n_features = int(n_features)
        self.input_col = input_col
        self.vec = HashingVectorizer(n_features=self.n_features, alternate_sign=False, norm=None)
    def fit(self, X, y=None): return self
    def transform(self, X):
        if isinstance(X, pd.DataFrame): X = X.iloc[:, 0]
        X = pd.Series(X).fillna("").astype(str).values
        return self.vec.transform(X).toarray()
    def get_feature_names_out(self, input_features=None):
        base = re.sub(r'\W+', '_', str(self.input_col).lower())
        return np.array([f"{base}_hash_{i}" for i in range(self.n_features)])

def choose_feature_columns(df: pd.DataFrame, target: str) -> Tuple[List[str], List[str], List[str]]:
    cols = df.columns.tolist()
    # text
    text_cols = [c for c in cols if c.lower() == 'description' and c != target]
    # categorical
    cat_cols = [c for c in cols if df[c].dtype == 'object' and c not in text_cols]
    for c in ['Date', 'Time', 'Datetime', 'ts']:
        if c in cat_cols:
            cat_cols.remove(c)
    # numeric (include ALL numeric columns)
    num_cols = [c for c in cols if np.issubdtype(df[c].dtype, np.number) and c != target]
    return text_cols, cat_cols, num_cols

def get_feature_names_from_ct(ct: ColumnTransformer) -> List[str]:
    names = []
    for name, transformer, cols in ct.transformers_:
        if transformer == "drop" or cols is None or len(cols) == 0: continue
        if isinstance(transformer, Pipeline):
            if name == "text":
                step = transformer.named_steps.get("hash")
                try: names.extend(step.get_feature_names_out())
                except: names.extend([f"text_hash_{i}" for i in range(HASH_DIM)])
                continue
            if name == "cat" and "onehot" in transformer.named_steps:
                ohe = transformer.named_steps["onehot"]
                try: ohe_names = ohe.get_feature_names_out(cols)
                except:
                    try: ohe_names = ohe.get_feature_names(cols)
                    except:
                        ohe_names = []
                        if hasattr(ohe,"categories_"):
                            for c, cat_vals in zip(cols, ohe.categories_):
                                for v in cat_vals: ohe_names.append(f"{c}_{v}")
                        else:
                            ohe_names = [f"cat__{c}" for c in cols]
                names.extend(list(ohe_names)); continue
            if name == "num":
                names.extend([f"{c}" for c in cols]); continue
            names.extend([f"{name}__{c}" for c in cols])
        else:
            try:
                direct_names = transformer.get_feature_names_out(cols)
                names.extend(direct_names.tolist())
            except:
                names.extend([f"{name}__{c}" for c in cols])
    return names

def safe_show(figpath: str, title: Optional[str] = None):
    try: plt.tight_layout()
    except: pass
    plt.savefig(figpath, bbox_inches="tight", dpi=150); plt.close()
    img = plt.imread(figpath); plt.figure(figsize=(8,6)); plt.imshow(img); plt.axis('off')
    if title: plt.title(title); plt.show()

def compute_class_threshold(y_vals: np.ndarray, strategy: str = "median", q: float = 0.5) -> float:
    y_vals = np.asarray(y_vals, dtype=float)
    if strategy == "median":
        return float(np.median(y_vals))
    elif strategy == "quantile":
        q = float(min(max(q, 0.0), 1.0))
        return float(np.quantile(y_vals, q))
    else:
        raise ValueError("CLASS_THRESHOLD_STRATEGY must be 'median' or 'quantile'")

def compute_vehicle_thresholds(train_df: pd.DataFrame,
                               strategy: str,
                               q: float,
                               min_n: int,
                               target_col: str = TARGET,
                               vehicle_col: str = "Vehicle") -> Dict[str, float]:
    """
    Returns a dict {vehicle -> threshold}. Vehicles with < min_n samples
    (or missing vehicle) fall back to the GLOBAL threshold under key '__GLOBAL__'.
    """
    thr_global = compute_class_threshold(train_df[target_col].values, strategy, q)
    out = {"__GLOBAL__": thr_global}
    if vehicle_col not in train_df.columns:
        return out

    grp = train_df.dropna(subset=[target_col]).groupby(vehicle_col)
    for v, g in grp:
        if len(g) >= min_n:
            out[str(v)] = compute_class_threshold(g[target_col].values, strategy, q)
    return out

def apply_vehicle_thresholds(df_like: pd.DataFrame,
                             preds: np.ndarray,
                             thresholds: Dict[str, float],
                             vehicle_col: str = "Vehicle",
                             target_col: str = TARGET) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Produces binary true/pred arrays using per-vehicle thresholds.
    Returns (y_true_bin, y_pred_bin, used_thresholds_per_row)
    """
    thr_global = thresholds.get("__GLOBAL__", compute_class_threshold(df_like[target_col].values, "median", 0.5))
    veh_series = df_like[vehicle_col].astype(str) if vehicle_col in df_like.columns else pd.Series(["__NA__"]*len(df_like), index=df_like.index)
    used_thr = np.array([thresholds.get(v, thr_global) for v in veh_series])
    y_true_bin = (df_like[target_col].values >= used_thr).astype(int)
    y_pred_bin = (preds >= used_thr).astype(int)
    return y_true_bin, y_pred_bin, used_thr

# --------------------------
# LOAD
# --------------------------
csv_path = DEFAULT_CSV_PATH
if not os.path.exists(csv_path):
    try:
        from google.colab import files  # type: ignore
        print("[Info] Please upload your final_dataset.csv")
        uploaded = files.upload()
        for k in uploaded.keys():
            if k.lower().endswith(".csv"): csv_path = f"/content/{k}"; break
        assert os.path.exists(csv_path), "No CSV file detected. Please upload your dataset."
    except Exception as e:
        raise FileNotFoundError("Upload your CSV via the Colab file widget or set csv_path.") from e

df = pd.read_csv(csv_path)
print(f"Loaded: {csv_path}")
print("Shape:", df.shape)
print("Columns:", list(df.columns))

# CLEAN
df = drop_slope_elevation(df)

speed_like = [c for c in df.columns if ("speed" in c.lower())]
for c in speed_like:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["timestamp"] = build_timestamp(df)

num_like = [
    'Temperature (C)','Humidity (%)','Precipitation (mm)','Wind Speed (km/h)',
    'Latitude','Longitude','hour','dow','is_weekend','is_wet','lat_bin','lon_bin',
    'is_speed_reason','timestamp'
]
for col in num_like:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

if TARGET not in df.columns:
    raise ValueError(f"Target '{TARGET}' not found in dataset.")
df = df[~df[TARGET].isna()].copy()
print(f"[Info] Rows after dropping missing target: {df.shape[0]}")

missing = df.isna().mean().sort_values(ascending=False).head(25)
plt.figure(figsize=(10,6)); missing.plot(kind="bar")
plt.title("Top-25 Missingness by Column"); plt.ylabel("Fraction Missing")
safe_show(os.path.join(OUT_DIR, "missingness_top25.png"))

plt.figure(figsize=(8,5))
plt.hist(df[TARGET].dropna().values, bins=40)
plt.title(f"Distribution of {TARGET}"); plt.xlabel(TARGET); plt.ylabel("Count")
safe_show(os.path.join(OUT_DIR, f"dist_{TARGET}.png"))

if "hour" in df.columns:
    plt.figure(figsize=(9,5))
    df["hour"].dropna().astype(int).value_counts().sort_index().plot(kind="bar")
    plt.title("Incidents by Hour of Day"); plt.xlabel("Hour"); plt.ylabel("Count")
    safe_show(os.path.join(OUT_DIR, "incidents_by_hour.png"))

if "dow" in df.columns:
    plt.figure(figsize=(9,5))
    df["dow"].dropna().astype(int).value_counts().sort_index().plot(kind="bar")
    plt.title("Incidents by Day of Week (0=Mon ... 6=Sun)"); plt.xlabel("Day"); plt.ylabel("Count")
    safe_show(os.path.join(OUT_DIR, "incidents_by_dow.png"))

if ("Longitude" in df.columns) and ("Latitude" in df.columns):
    plt.figure(figsize=(6,6))
    plt.scatter(df["Longitude"], df["Latitude"], s=8, alpha=0.8)
    plt.title("Incident Locations (Longitude vs Latitude)")
    plt.xlabel("Longitude"); plt.ylabel("Latitude")
    safe_show(os.path.join(OUT_DIR, "geo_scatter.png"))

num_cols_all = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
corr_subset = [c for c in num_cols_all if c in [
    'Temperature (C)','Humidity (%)','Precipitation (mm)','Wind Speed (km/h)',
    'Latitude','Longitude','hour','dow','is_weekend','is_wet','lat_bin','lon_bin',
    'is_speed_reason','timestamp', TARGET
]]
if len(corr_subset) >= 3:
    corr = df[corr_subset].corr(numeric_only=True)
    plt.figure(figsize=(9,7))
    im = plt.imshow(corr, cmap="coolwarm", vmin=-1, vmax=1)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks(range(len(corr_subset)), corr_subset, rotation=45, ha="right")
    plt.yticks(range(len(corr_subset)), corr_subset)
    plt.title("Correlation Heatmap (subset)")
    safe_show(os.path.join(OUT_DIR, "correlation_heatmap.png"))

# FEATURES + PREPROCESS
text_cols, cat_cols, num_cols = choose_feature_columns(df, TARGET)
print("\nSelected feature groups:")
print("  Text:", text_cols if text_cols else "None")
print("  Categorical:", cat_cols if cat_cols else "None")
print("  Numeric:", num_cols if num_cols else "None")

text_transformer = Pipeline(steps=[
    ("hash", DenseHashingVectorizer(n_features=HASH_DIM,
                                    input_col=text_cols[0] if text_cols else "text"))
]) if text_cols else "drop"

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
]) if cat_cols else "drop"

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
]) if num_cols else "drop"

preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, text_cols),
        ("cat" , categorical_transformer, cat_cols),
        ("num" , numeric_transformer, num_cols),
    ],
    remainder="drop"
)

# SPLIT (chronological)
use_time_split = df["timestamp"].notna().any()
if use_time_split: df = df.sort_values("timestamp")
else: print("[Warn] No usable timestamp found; using order-based split.")

train_size = int(0.8 * len(df))
train_df = df.iloc[:train_size].copy()
test_df  = df.iloc[train_size:].copy()

X_train = train_df[text_cols + cat_cols + num_cols]
y_train = train_df[TARGET].values
X_test  = test_df[text_cols + cat_cols + num_cols]
y_test  = test_df[TARGET].values

print(f"\nTrain size: {len(X_train)} | Test size: {len(X_test)}")
if use_time_split:
    print(f"Train time span: {pd.to_datetime(train_df['timestamp'], unit='s', errors='coerce').min()} -> "
          f"{pd.to_datetime(train_df['timestamp'], unit='s', errors='coerce').max()}")
    print(f"Test  time span: {pd.to_datetime(test_df['timestamp'], unit='s', errors='coerce').min()} -> "
          f"{pd.to_datetime(test_df['timestamp'], unit='s', errors='coerce').max()}")

# MODEL + TUNING
base_xgb = XGBRegressor(
    n_estimators=700, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8,
    reg_lambda=1.0, reg_alpha=0.0, min_child_weight=1.0,
    objective="reg:squarederror", tree_method="hist",
    importance_type="gain", random_state=RANDOM_STATE, n_jobs=-1
)

pipeline = Pipeline(steps=[("pre", preprocessor), ("xgb", base_xgb)])

if DO_TUNE:
    print("\n[Info] Starting randomized hyperparameter search...")
    cv = TimeSeriesSplit(n_splits=CV_SPLITS) if use_time_split else KFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    param_distributions = {
        "xgb__n_estimators":   np.linspace(300, 1200, 10, dtype=int),
        "xgb__learning_rate":  np.linspace(0.01, 0.2, 20),
        "xgb__max_depth":      np.arange(3, 11),
        "xgb__subsample":      np.linspace(0.6, 1.0, 9),
        "xgb__colsample_bytree": np.linspace(0.6, 1.0, 9),
        "xgb__min_child_weight": np.linspace(0.5, 8.0, 16),
        "xgb__reg_lambda":     np.linspace(0.0, 5.0, 11),
        "xgb__reg_alpha":      np.linspace(0.0, 1.0, 11),
    }
    rs = RandomizedSearchCV(
        estimator=pipeline, param_distributions=param_distributions,
        n_iter=N_ITER_TUNE, scoring="r2", cv=cv, verbose=1,
        random_state=RANDOM_STATE, n_jobs=-1
    )
    rs.fit(X_train, y_train)
    best_model = rs.best_estimator_
    clean_params = {k: (v.item() if hasattr(v, "item") else v) for k, v in rs.best_params_.items()}
    print("\n[Info] Best CV R^2:", float(rs.best_score_))
    print("[Info] Best Params:", clean_params)
else:
    print("\n[Info] Training base XGBoost model (no tuning)...")
    best_model = pipeline.fit(X_train, y_train)

# EVALUATION (regression)
preds = best_model.predict(X_test)
r2  = float(r2_score(y_test, preds))
mae = float(mean_absolute_error(y_test, preds))
rmse = float(np.sqrt(mean_squared_error(y_test, preds)))

print("\n=== Test Metrics (SPI_smoothed regression, XGBoost) ===")
print(f"R^2   : {r2:.6f}")
print(f"MAE   : {mae:.6f}")
print(f"RMSE  : {rmse:.6f}")

# Residuals vs Pred
residuals = y_test - preds
plt.figure(figsize=(7,5))
plt.scatter(preds, residuals, s=12, alpha=0.7)
plt.axhline(0, color="gray", linestyle="--", linewidth=1)
plt.xlabel("Predicted SPI_smoothed"); plt.ylabel("Residual (True - Pred)")
plt.title("Residuals vs Predicted")
safe_show(os.path.join(OUT_DIR, "residuals_vs_pred.png"))

# Residuals distribution
plt.figure(figsize=(7,5))
plt.hist(residuals, bins=40)
plt.title("Residuals Distribution"); plt.xlabel("Residual"); plt.ylabel("Count")
safe_show(os.path.join(OUT_DIR, "residuals_hist.png"))

# Parity plot (y = x)
plt.figure(figsize=(6,6))
plt.scatter(y_test, preds, s=12, alpha=0.7)
lims = [min(y_test.min(), preds.min()), max(y_test.max(), preds.max())]
plt.plot(lims, lims, 'k--', linewidth=1)
plt.xlabel("Actual SPI_smoothed"); plt.ylabel("Predicted SPI_smoothed")
plt.title("Parity Plot (Pred vs Actual)")
safe_show(os.path.join(OUT_DIR, "parity_plot.png"))

# Time series residuals (if timestamp)
if test_df["timestamp"].notna().any():
    ts_ser = pd.to_datetime(test_df["timestamp"], unit='s', errors='coerce')
    plt.figure(figsize=(10,4))
    plt.plot(ts_ser, residuals, marker='o', linestyle='-', linewidth=1)
    plt.axhline(0, color="gray", linestyle="--", linewidth=1)
    plt.title("Residuals over Time (Test Set)")
    plt.xlabel("Time"); plt.ylabel("Residual")
    plt.xticks(rotation=25)
    safe_show(os.path.join(OUT_DIR, "residuals_over_time.png"))

# Learning curve (optional)
max_for_lc = min(len(X_train), 5000)
if max_for_lc >= 200:
    X_lc = X_train.iloc[:max_for_lc]; y_lc = y_train[:max_for_lc]
    cv_lc = TimeSeriesSplit(n_splits=CV_SPLITS) if use_time_split else KFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    train_sizes, train_scores, val_scores = learning_curve(
        estimator=best_model, X=X_lc, y=y_lc, cv=cv_lc, scoring="r2",
        train_sizes=np.linspace(0.1, 1.0, 8), n_jobs=-1, shuffle=False
    )
    plt.figure(figsize=(8,5))
    plt.plot(train_sizes, train_scores.mean(axis=1), marker="o", label="Train R^2")
    plt.plot(train_sizes, val_scores.mean(axis=1), marker="s", label="CV R^2")
    plt.xlabel("Training samples"); plt.ylabel("R^2"); plt.title("Learning Curve (XGBoost)")
    plt.legend()
    safe_show(os.path.join(OUT_DIR, "learning_curve.png"))

# THRESHOLDS + CLASSIFICATION METRICS (GLOBAL & PER-VEHICLE)
# Global baseline threshold (train-only)
thr_global = compute_class_threshold(y_train, CLASS_THRESHOLD_STRATEGY, CLASS_THRESHOLD_Q)

# Per-vehicle thresholds (train-only)
vehicle_thresholds = compute_vehicle_thresholds(
    train_df, CLASS_THRESHOLD_STRATEGY, CLASS_THRESHOLD_Q,
    MIN_TRAIN_RECORDS_PER_VEHICLE, TARGET, "Vehicle"
)

# Save thresholds
thr_rows = [{"Vehicle": k, "threshold": v} for k, v in vehicle_thresholds.items() if k != "__GLOBAL__"]
thr_rows.append({"Vehicle": "__GLOBAL__", "threshold": vehicle_thresholds["__GLOBAL__"]})
pd.DataFrame(thr_rows).to_csv(os.path.join(OUT_DIR, "vehicle_thresholds.csv"), index=False)

# Apply thresholds on test set
if CLASS_THRESHOLD_PER_VEHICLE:
    y_test_cls, y_pred_cls, used_thr_vec = apply_vehicle_thresholds(
        test_df, preds, vehicle_thresholds, vehicle_col="Vehicle", target_col=TARGET
    )
    thr_used_label = "per-vehicle"
else:
    used_thr_vec = np.full_like(preds, fill_value=thr_global, dtype=float)
    y_test_cls = (y_test >= thr_global).astype(int)
    y_pred_cls = (preds  >= thr_global).astype(int)
    thr_used_label = "global"

# Overall metrics
acc = float(accuracy_score(y_test_cls, y_pred_cls))
prec, rec, f1, _ = precision_recall_fscore_support(y_test_cls, y_pred_cls, average="binary", zero_division=0)
prec = float(prec); rec = float(rec); f1 = float(f1)

print(f"\n=== Test Classification Metrics (thresholded SPI for alerts; {thr_used_label}) ===")
if thr_used_label == "global":
    print(f"Global Threshold used on SPI (train-derived): {thr_global:.6f}  (strategy={CLASS_THRESHOLD_STRATEGY}, q={CLASS_THRESHOLD_Q})")
else:
    print(f"Per-vehicle thresholds used (see vehicle_thresholds.csv); global fallback = {vehicle_thresholds['__GLOBAL__']:.6f}")
print(f"Accuracy : {acc:.6f}")
print(f"Precision: {prec:.6f}")
print(f"Recall   : {rec:.6f}")
print(f"F1-score : {f1:.6f}")

# Per-vehicle metrics table
veh_list = test_df["Vehicle"].astype(str) if "Vehicle" in test_df.columns else pd.Series(["__NA__"]*len(test_df))
per_vehicle_rows = []
for v in sorted(veh_list.unique()):
    idx = (veh_list == v).values
    if idx.sum() == 0: continue
    acc_v = float(accuracy_score(y_test_cls[idx], y_pred_cls[idx]))
    p_v, r_v, f_v, _ = precision_recall_fscore_support(y_test_cls[idx], y_pred_cls[idx], average="binary", zero_division=0)
    thr_v = vehicle_thresholds.get(v, vehicle_thresholds["__GLOBAL__"])
    per_vehicle_rows.append({
        "Vehicle": v,
        "n_test": int(idx.sum()),
        "threshold_used": float(thr_v),
        "accuracy": float(acc_v),
        "precision": float(p_v),
        "recall": float(r_v),
        "f1": float(f_v),
    })
per_vehicle_df = pd.DataFrame(per_vehicle_rows).sort_values(["Vehicle"])
per_vehicle_csv = os.path.join(OUT_DIR, "classification_metrics_per_vehicle.csv")
per_vehicle_df.to_csv(per_vehicle_csv, index=False)
print("\nPer-vehicle alert metrics saved to:", per_vehicle_csv)

# FEATURE IMPORTANCE + SHAP
ct = best_model.named_steps["pre"]
feat_names = get_feature_names_from_ct(ct)

xgb_est = best_model.named_steps["xgb"]
importances = getattr(xgb_est, "feature_importances_", None)
if importances is not None and len(importances) == len(feat_names):
    order = np.argsort(importances)[::-1]
    k = min(TOP_FEATS, len(order))
    top_features = [(feat_names[i], float(importances[i])) for i in order[:k]]
    pd.DataFrame(top_features, columns=["feature", "importance"]).to_csv(
        os.path.join(OUT_DIR, "top_features.csv"), index=False
    )
    plt.figure(figsize=(10,7))
    names_plot = [f for f, _ in top_features[:25]][::-1]
    vals_plot  = [v for _, v in top_features[:25]][::-1]
    plt.barh(names_plot, vals_plot)
    plt.title("Top Feature Importances (XGBoost, gain)")
    plt.xlabel("Importance (gain)")
    safe_show(os.path.join(OUT_DIR, "feature_importances_top25.png"))
else:
    print("[Warn] Could not align feature importances with names; skipping plot.")

# SHAP (sampled)
try:
    X_test_trans = ct.transform(X_test)
    rng = np.random.default_rng(RANDOM_STATE)
    if X_test_trans.shape[0] > SHAP_SAMPLE:
        idx = rng.choice(X_test_trans.shape[0], SHAP_SAMPLE, replace=False)
        X_shap = X_test_trans[idx]
    else:
        X_shap = X_test_trans
    explainer = shap.TreeExplainer(xgb_est)
    shap_values = explainer.shap_values(X_shap)
    shap.summary_plot(shap_values, X_shap, feature_names=feat_names, show=False)
    safe_show(os.path.join(OUT_DIR, "shap_summary.png"))
    shap.summary_plot(shap_values, X_shap, feature_names=feat_names, plot_type="bar", show=False)
    safe_show(os.path.join(OUT_DIR, "shap_bar.png"))
except Exception as e:
    print(f"[Warn] SHAP explanation skipped: {e}")

# SAVE ARTIFACTS
model_path = os.path.join(MODEL_DIR, "xgb_vehicle_specific_risk.pkl")
joblib.dump(best_model, model_path)
print(f"\n[Info] Model saved to: {model_path}")

# Core regression metrics
metrics = {
    "dataset_path": csv_path,
    "n_train": int(len(X_train)),
    "n_test": int(len(X_test)),
    "target": TARGET,
    "model": "XGBRegressor",
    "tuned": bool(DO_TUNE),
    "test_metrics": {"r2": r2, "mae": mae, "rmse": rmse}
}
with open(os.path.join(OUT_DIR, "metrics.json"), "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2)
print("[Info] Metrics saved to:", os.path.join(OUT_DIR, "metrics.json"))

# Save overall classification metrics (for the alerting use-case)
cls_metrics = {
    "threshold_mode": "per-vehicle" if CLASS_THRESHOLD_PER_VEHICLE else "global",
    "global_threshold": float(thr_global),
    "strategy": CLASS_THRESHOLD_STRATEGY,
    "q": CLASS_THRESHOLD_Q,
    "accuracy": acc,
    "precision": prec,
    "recall": rec,
    "f1": f1
}
with open(os.path.join(OUT_DIR, "classification_metrics.json"), "w", encoding="utf-8") as f:
    json.dump(cls_metrics, f, indent=2)
print("[Info] Classification metrics (overall) saved to:", os.path.join(OUT_DIR, "classification_metrics.json"))

# Save per-row predictions + flags + threshold used
pred_df = test_df.copy()
pred_df["SPI_true"] = y_test
pred_df["SPI_pred"] = preds
pred_df["residual"] = pred_df["SPI_true"] - pred_df["SPI_pred"]
pred_df["thr_used"] = used_thr_vec
pred_df["is_high_true"] = y_test_cls
pred_df["is_high_pred"] = y_pred_cls
keep_cols = ["SPI_true","SPI_pred","residual","thr_used","is_high_true","is_high_pred",
             "Vehicle","Place","Reason","Position","segment_id","timestamp"]
keep_cols = [c for c in keep_cols if c in pred_df.columns]
pred_csv = os.path.join(OUT_DIR, "predictions.csv")
pred_df[keep_cols].to_csv(pred_csv, index=False)
print("[Info] Predictions saved to:", pred_csv)

# Save per-vehicle metrics JSON too
with open(os.path.join(OUT_DIR, "classification_metrics_per_vehicle.json"), "w", encoding="utf-8") as f:
    json.dump(per_vehicle_rows, f, indent=2)
print("[Info] Per-vehicle classification metrics saved to: classification_metrics_per_vehicle.json")

# One-sample preview
if len(pred_df) > 0:
    n_preview = 3
    sample = pred_df.iloc[:n_preview]

    for i, row in sample.iterrows():
        print("\nExample prediction:")
        print("  Vehicle :", row.get("Vehicle", "NA"))
        print("  Place   :", row.get("Place", "NA"))
        print("  Reason  :", row.get("Reason", "NA"))
        print("  Position:", row.get("Position", "NA"))
        print("  Time    :", row.get("Datetime", row.get("Date", "NA")))

        # Model outputs
        print("  Predicted SPI_smoothed:", round(float(row["SPI_pred"]), 6))
        print("  Actual    SPI_smoothed:", round(float(row["SPI_true"]), 6))
        print("  Thr used :", round(float(row["thr_used"]), 6))
        print("  HighRisk (pred/true):", int(row["is_high_pred"]), "/", int(row["is_high_true"]))

        # Optional numeric context if available
        for feat in [
            "Temperature (C)",
            "Humidity (%)",
            "Precipitation (mm)",
            "Wind Speed (km/h)",
            "hour",
            "dow",
            "is_wet",
        ]:
            if feat in row.index:
                print(f"  {feat}: {row[feat]}")

    print("\n=== DONE ===")
    print(f"Outputs directory: {OUT_DIR}")
    print(f"Model directory  : {MODEL_DIR}")


[Info] Please upload your final_dataset.csv


Saving final_dataset.csv to final_dataset.csv
Loaded: /content/final_dataset.csv
Shape: (315, 24)
Columns: ['Date', 'Time', 'Vehicle', 'Place', 'Reason', 'Position', 'Description', 'Datetime', 'Temperature (C)', 'Humidity (%)', 'Precipitation (mm)', 'Wind Speed (km/h)', 'Latitude', 'Longitude', 'ts', 'hour', 'dow', 'is_weekend', 'is_wet', 'lat_bin', 'lon_bin', 'segment_id', 'is_speed_reason', 'SPI_smoothed']
[Info] Rows after dropping missing target: 315

Selected feature groups:
  Text: ['Description']
  Categorical: ['Vehicle', 'Place', 'Reason', 'Position', 'segment_id']
  Numeric: ['Temperature (C)', 'Humidity (%)', 'Precipitation (mm)', 'Wind Speed (km/h)', 'Latitude', 'Longitude', 'hour', 'dow', 'is_weekend', 'is_wet', 'lat_bin', 'lon_bin', 'is_speed_reason', 'timestamp']

Train size: 252 | Test size: 63
Train time span: 2020-01-06 08:00:00 -> 2023-11-29 07:20:00
Test  time span: 2023-12-06 14:35:00 -> 2025-03-28 15:40:00

[Info] Starting randomized hyperparameter search...
Fitti

In [3]:
# -*- coding: utf-8 -*-
# ================================================================
# Test saved pipeline on the SAME dataset, using the LAST 63 rows
# - Model : /content/models/xgb_vehicle_specific_risk.pkl
# - Data  : /content/final_dataset.csv
# - Uses  : Per-vehicle thresholds if /content/outputs/vehicle_thresholds.csv exists,
#           otherwise derives thresholds from the TRAIN portion (all rows except last 63).
# - Outputs:
#     /content/outputs/predictions_from_pkl.csv
#     /content/outputs/classification_metrics_overall.json
#     /content/outputs/classification_metrics_per_vehicle.csv
# - Console preview shows a few sample rows per common vehicle type.
# ================================================================

import os, warnings
from typing import List, Tuple, Dict
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    accuracy_score, precision_recall_fscore_support
)

MODEL_PATH = "/content/models/xgb_vehicle_specific_risk.pkl"
CSV_PATH   = "/content/final_dataset.csv"
OUT_DIR    = "/content/outputs"
THR_CSV    = os.path.join(OUT_DIR, "vehicle_thresholds.csv")  # produced by training script
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------- helpers (mirror training rules) ----------------
def build_timestamp(df: pd.DataFrame) -> pd.Series:
    ts = None
    if 'ts' in df.columns:
        t = pd.to_datetime(df['ts'], errors="coerce")
        if t.notna().any(): ts = t
    if ts is None and 'Datetime' in df.columns:
        t = pd.to_datetime(df['Datetime'], errors="coerce")
        if t.notna().any(): ts = t
    if ts is None and ('Date' in df.columns or 'Time' in df.columns):
        if 'Date' in df.columns and 'Time' in df.columns:
            t = pd.to_datetime(df['Date'].astype(str) + " " + df['Time'].astype(str), errors="coerce")
        else:
            t = pd.to_datetime(df.get('Date', pd.NaT), errors="coerce")
        ts = t
    if ts is None:
        ts = pd.Series([pd.NaT]*len(df), index=df.index)
    return (ts.astype('int64') // 10**9).astype('float')

def drop_slope_elevation(df: pd.DataFrame) -> pd.DataFrame:
    to_drop = [c for c in df.columns if ('slope' in c.lower()) or ('elev' in c.lower())]
    return df.drop(columns=to_drop, errors="ignore")

def expected_input_columns_from_pipeline(pipeline) -> List[str]:
    ct = pipeline.named_steps["pre"]
    inputs = []
    for name, transformer, cols in ct.transformers_:
        if transformer == "drop" or cols is None:
            continue
        if isinstance(cols, list):
            inputs.extend(cols)
        elif isinstance(cols, (np.ndarray, pd.Index)):
            inputs.extend(list(cols))
        else:
            try:
                inputs.append(cols)
            except:
                pass
    seen, ordered = set(), []
    for c in inputs:
        if c not in seen:
            seen.add(c); ordered.append(c)
    return ordered

def compute_class_threshold(y_vals: np.ndarray, strategy: str = "median", q: float = 0.5) -> float:
    y_vals = np.asarray(y_vals, dtype=float)
    if strategy == "median":
        return float(np.median(y_vals))
    q = float(min(max(q, 0.0), 1.0))
    return float(np.quantile(y_vals, q))

def compute_vehicle_thresholds_from_train(train_df: pd.DataFrame,
                                          strategy: str = "median",
                                          q: float = 0.5,
                                          min_n: int = 8,
                                          target_col: str = "SPI_smoothed",
                                          vehicle_col: str = "Vehicle") -> Dict[str, float]:
    thr_global = compute_class_threshold(train_df[target_col].values, strategy, q)
    out = {"__GLOBAL__": thr_global}
    if vehicle_col not in train_df.columns:
        return out
    grp = train_df.dropna(subset=[target_col]).groupby(vehicle_col)
    for v, g in grp:
        if len(g) >= min_n:
            out[str(v)] = compute_class_threshold(g[target_col].values, strategy, q)
    return out

def load_vehicle_thresholds_or_build(train_df: pd.DataFrame) -> Dict[str, float]:
    """
    Priority:
      1) Load thresholds saved by training at OUT_DIR/vehicle_thresholds.csv
      2) Else, derive from train_df (all rows except the last 63), per-vehicle with global fallback
    """
    if os.path.exists(THR_CSV):
        df_thr = pd.read_csv(THR_CSV)
        d = {str(row["Vehicle"]): float(row["threshold"]) for _, row in df_thr.iterrows() if row["Vehicle"] != "__GLOBAL__"}
        glob = df_thr.loc[df_thr["Vehicle"] == "__GLOBAL__", "threshold"]
        d["__GLOBAL__"] = float(glob.iloc[0]) if len(glob) else compute_class_threshold(train_df["SPI_smoothed"].values, "median", 0.5)
        print(f"[OK] Loaded vehicle thresholds from {THR_CSV}")
        return d
    print("[Warn] vehicle_thresholds.csv not found. Building thresholds from training portion...")
    return compute_vehicle_thresholds_from_train(train_df, strategy="median", q=0.5, min_n=8)

def apply_vehicle_thresholds(df_like: pd.DataFrame,
                             preds: np.ndarray,
                             thresholds: Dict[str, float],
                             vehicle_col: str = "Vehicle",
                             target_col: str = "SPI_smoothed") -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    thr_global = thresholds.get("__GLOBAL__", compute_class_threshold(df_like[target_col].values, "median", 0.5))
    veh_series = df_like[vehicle_col].astype(str) if vehicle_col in df_like.columns else pd.Series(["__NA__"]*len(df_like), index=df_like.index)
    used_thr = np.array([thresholds.get(v, thr_global) for v in veh_series])
    y_true_bin = (df_like[target_col].values >= used_thr).astype(int)
    y_pred_bin = (preds >= used_thr).astype(int)
    return y_true_bin, y_pred_bin, used_thr

# ---------------- load model ----------------
assert os.path.exists(MODEL_PATH), f"Model not found: {MODEL_PATH}"
model = joblib.load(MODEL_PATH)
print("[OK] Loaded model:", MODEL_PATH)

# ---------------- load data ----------------
if not os.path.exists(CSV_PATH):
    try:
        from google.colab import files  # type: ignore
        print("[Info] Please upload your final_dataset.csv")
        uploaded = files.upload()
        for k in uploaded.keys():
            if k.lower().endswith(".csv"):
                CSV_PATH = f"/content/{k}"
                break
        assert os.path.exists(CSV_PATH), "No CSV detected."
    except Exception as e:
        raise FileNotFoundError("Upload your CSV or set CSV_PATH.") from e

df_raw = pd.read_csv(CSV_PATH)
print(f"[OK] Loaded data: {CSV_PATH} | shape={df_raw.shape}")

# ---------------- light cleaning (as in training) ----------------
df = drop_slope_elevation(df_raw.copy())
df["timestamp"] = build_timestamp(df)

num_like = [
    'Temperature (C)','Humidity (%)','Precipitation (mm)','Wind Speed (km/h)',
    'Latitude','Longitude','hour','dow','is_weekend','is_wet','lat_bin','lon_bin',
    'is_speed_reason','timestamp'
]
for col in num_like:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

if "SPI_smoothed" not in df.columns:
    raise ValueError("Column 'SPI_smoothed' not found in the dataset.")

# ---------------- chronological split ----------------
if df["timestamp"].notna().any():
    df_sorted = df.sort_values("timestamp")
else:
    print("[Warn] No usable timestamps; using original row order.")
    df_sorted = df.copy()

n_test = 63
assert len(df_sorted) >= n_test, f"Dataset has only {len(df_sorted)} rows; need at least {n_test}."
train_df = df_sorted.iloc[:-n_test].copy()
test_df  = df_sorted.iloc[-n_test:].copy()
print(f"[OK] Selected last {n_test} rows as test set. Train={len(train_df)} Test={len(test_df)}")

# ---------------- align features ----------------
expected_cols = expected_input_columns_from_pipeline(model)
present = [c for c in expected_cols if c in test_df.columns]
missing = [c for c in expected_cols if c not in test_df.columns]
if missing:
    print("[Warn] Missing expected columns (imputers/one-hot usually handle this):", missing)

X_test = test_df[present].copy()
y_test = test_df["SPI_smoothed"].values

# ---------------- predict ----------------
pred = model.predict(X_test)

# ---------------- regression metrics ----------------
r2   = float(r2_score(y_test, pred))
mae  = float(mean_absolute_error(y_test, pred))
rmse = float(np.sqrt(mean_squared_error(y_test, pred)))
print("\n=== Regression metrics on last 63 ===")
print(f"R^2   : {r2:.6f}")
print(f"MAE   : {mae:.6f}")
print(f"RMSE  : {rmse:.6f}")

# ---------------- thresholds + alert metrics (per-vehicle) ----------------
vehicle_thresholds = load_vehicle_thresholds_or_build(train_df)

y_true_bin, y_pred_bin, thr_used_vec = apply_vehicle_thresholds(
    test_df, pred, vehicle_thresholds, vehicle_col="Vehicle", target_col="SPI_smoothed"
)

acc = float(accuracy_score(y_true_bin, y_pred_bin))
prec, rec, f1, _ = precision_recall_fscore_support(y_true_bin, y_pred_bin, average="binary", zero_division=0)
print("\n=== Alert metrics (per-vehicle thresholds) on last 63 ===")
print(f"Accuracy : {acc:.6f}")
print(f"Precision: {float(prec):.6f}")
print(f"Recall   : {float(rec):.6f}")
print(f"F1-score : {float(f1):.6f}")

# Per-vehicle breakdown
veh_series = test_df["Vehicle"].astype(str) if "Vehicle" in test_df.columns else pd.Series(["__NA__"]*len(test_df), index=test_df.index)
rows = []
for v in sorted(veh_series.unique()):
    idx = (veh_series == v).values
    if idx.sum() == 0:
        continue
    acc_v = float(accuracy_score(y_true_bin[idx], y_pred_bin[idx]))
    p_v, r_v, f_v, _ = precision_recall_fscore_support(y_true_bin[idx], y_pred_bin[idx], average="binary", zero_division=0)
    thr_v = vehicle_thresholds.get(v, vehicle_thresholds["__GLOBAL__"])
    rows.append({
        "Vehicle": v,
        "n_test": int(idx.sum()),
        "threshold_used": float(thr_v),
        "accuracy": float(acc_v),
        "precision": float(p_v),
        "recall": float(r_v),
        "f1": float(f_v),
    })

per_vehicle_df = pd.DataFrame(rows).sort_values(["n_test","Vehicle"], ascending=[False, True])
per_vehicle_csv = os.path.join(OUT_DIR, "classification_metrics_per_vehicle.csv")
per_vehicle_df.to_csv(per_vehicle_csv, index=False)
print("\n[OK] Per-vehicle metrics saved to:", per_vehicle_csv)
print(per_vehicle_df.to_string(index=False))

# ---------------- save enriched predictions ----------------
out_df = pd.DataFrame({
    "SPI_true": y_test.astype(float),
    "SPI_pred": pred.astype(float),
    "residual": (y_test - pred).astype(float),
    "thr_used": thr_used_vec.astype(float),
    "is_high_true": y_true_bin.astype(int),
    "is_high_pred": y_pred_bin.astype(int),
})

# Add time & context columns if present
for c in ["timestamp", "Datetime", "Date", "Time",
          "Vehicle","Place","Reason","Position","segment_id","lat_bin","lon_bin",
          "Latitude","Longitude","hour","dow","is_weekend","is_wet","is_speed_reason"]:
    if c in test_df.columns:
        out_df[c] = test_df[c].values

save_path = os.path.join(OUT_DIR, "predictions_from_pkl.csv")
out_df.to_csv(save_path, index=False)
print(f"\n[OK] Saved predictions (with thresholds & flags) to:\n  {save_path}")

# ---------------- also save overall alert metrics ----------------
overall_json = {
    "accuracy": acc,
    "precision": float(prec),
    "recall": float(rec),
    "f1": float(f1),
    "n_test": int(len(test_df)),
    "note": "Per-vehicle thresholds with global fallback"
}
import json
with open(os.path.join(OUT_DIR, "classification_metrics_overall.json"), "w", encoding="utf-8") as f:
    json.dump(overall_json, f, indent=2)
print("[OK] Overall alert metrics saved to: classification_metrics_overall.json")

# ---------------- quick console preview by vehicle ----------------
print("\n=== Sample rows per common vehicle ===")
if "Vehicle" in out_df.columns:
    top_vs = out_df["Vehicle"].value_counts().head(3).index.tolist()
    for v in top_vs:
        sample = out_df[out_df["Vehicle"] == v].head(3)
        print(f"\nVehicle: {v} (showing up to 3 rows)")
        cols_show = [c for c in ["Date","Time","Datetime","segment_id","Place",
                                 "SPI_true","SPI_pred","thr_used","is_high_true","is_high_pred"] if c in sample.columns]
        print(sample[cols_show].to_string(index=False))
else:
    print("Vehicle column not found; skipped vehicle-wise samples.")


[OK] Loaded model: /content/models/xgb_vehicle_specific_risk.pkl
[OK] Loaded data: /content/final_dataset.csv | shape=(315, 24)
[OK] Selected last 63 rows as test set. Train=252 Test=63

=== Regression metrics on last 63 ===
R^2   : 0.652261
MAE   : 0.009836
RMSE  : 0.014979
[OK] Loaded vehicle thresholds from /content/outputs/vehicle_thresholds.csv

=== Alert metrics (per-vehicle thresholds) on last 63 ===
Accuracy : 0.761905
Precision: 0.900000
Recall   : 0.765957
F1-score : 0.827586

[OK] Per-vehicle metrics saved to: /content/outputs/classification_metrics_per_vehicle.csv
                    Vehicle  n_test  threshold_used  accuracy  precision   recall       f1
              Three Wheeler      14        0.350718  0.857143        1.0 0.846154 0.916667
                Motor Cycle       8        0.398337  1.000000        1.0 1.000000 1.000000
                        Car       7        0.350718  0.428571        1.0 0.428571 0.600000
                      Lorry       6        0.350718  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')